In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import arviz as az

from cmdstanpy import cmdstan_path, CmdStanModel

In [2]:
PROJECT_DIR = "/home/george/parts/"
print(cmdstan_path())

/home/george/.cmdstanpy/cmdstan-2.21.0


## Data

In [11]:
df = pd.read_csv("../data/tidy/parts.csv")

In [25]:
class_name = "analysis.ii"

friday_mask = (df.loc[:, "class"] == "analysis.ii") & (df.loc[:, "day"] == "fri")
friday_data = df[friday_mask].loc[:, "parts"].dropna().values
non_friday_data = df[~friday_mask].loc[:, "parts"].dropna().values
N = non_friday_data.size
M = friday_data.size

In [66]:
def run_stan_model(stan_file, data, sample=True, **kwargs):
    """ Convenience function to compile, sample and diagnose a Stan model. """
    model = CmdStanModel(stan_file=stan_file)
    model.compile()
    if sample:
        fit = model.sample(data=data, **kwargs)
        fit.diagnose()
        return model, fit
    else:
        return model

## Examine Prior Predictive Distribution

In [86]:
data = {
    "N": 100  # Number of iterations
}

model = run_stan_model(
    stan_file=os.path.join(PROJECT_DIR, "models", "_00-initial-prior_pred.stan"),
    data=data,
)

INFO:cmdstanpy:found newer exe file, not recompiling
INFO:cmdstanpy:compiled model file: /home/george/parts/models/_00-initial-prior_pred
INFO:cmdstanpy:start chain 1
INFO:cmdstanpy:start chain 2
INFO:cmdstanpy:finish chain 2
INFO:cmdstanpy:start chain 3
INFO:cmdstanpy:finish chain 1
INFO:cmdstanpy:start chain 4
INFO:cmdstanpy:finish chain 3
INFO:cmdstanpy:finish chain 4


RuntimeError: Error during sampling, chain 0 returned error code 78, chain 1 returned error code 78, chain 2 returned error code 78, chain 3 returned error code 78

In [85]:
model.generate_quantities(data=data, mcmc_sample=)

ValueError: mcmc_sample must be either CmdStanMCMC object or list of paths to sample csv_files

In [60]:
# TODO: figure out how to use arviz

cmdstanpy_data = az.from_cmdstanpy(
    posterior=fit,
    posterior_predictive="y_hat",
    log_likelihood="log_lik",
    coords={'school': np.arange(eight_school_data['J'])},
    dims={
        'theta': ['school'],
        'y': ['school'],
        'log_lik': ['school'],
        'y_hat': ['school'],
        'theta_tilde': ['school']
    }
)
cmdstanpy_data

KeyError: 'y'